<a href="https://colab.research.google.com/github/MauricioVazquezM/Skin_Cancer_Detection_Project/blob/memo/NOTEBOOKS%26CODE/ModeloXGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba de modelo XG Boost para metadatos

## Content

- Libraries
- Data preparation
- Model structure
- Tests

In [ ]:
# Librerías
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, make_scorer
from skimage.feature import hog
from skimage import io, exposure, img_as_float

### Data preparation

#### Metadata

In [ ]:
df = pd.read_csv('C:/Users/maria/OneDrive/Escritorio/ITAM/Mineria/cancer/df_datos.csv')
df.head(5)

,isic_id,target,age_approx,sex,clin_size_long_diam_mm,tbp_lv_areaMM2,tbp_lv_deltaLBnorm,tbp_lv_location_simple,tbp_lv_norm_border,tbp_lv_norm_color
0,ISIC_8140686,0,70.0,female,2.85,4.240946,6.484014,Left Leg,3.409160,1.608265
1,ISIC_8374103,0,65.0,female,2.09,2.514543,7.706197,Right Leg,2.239681,2.190920
2,ISIC_7054001,0,60.0,male,2.70,3.640458,5.753112,Torso Back,2.897936,1.175670
3,ISIC_7511473,0,55.0,female,6.50,26.158754,8.873213,Left Arm,3.962258,8.242360
4,ISIC_2831112,0,50.0,female,2.68,2.852318,5.052441,Torso Front,3.955003,0.000000


In [ ]:
# # Seguimos los pasos realizados durante el EDA - eliminación de variables

# # Ahora eliminaremos las columnas del dataframe que deban ser eliminadas con base
#   # en el análisis anterios (explican lo mismo), así como las variables que
#   # sabemos que no aportan nada o no nos servirán de nada
# df.drop(['tbp_lv_A',
# 'tbp_lv_Aext',
# 'tbp_lv_B',
# 'tbp_lv_Bext',
# 'tbp_lv_C',
# 'tbp_lv_Cext',
# 'tbp_lv_H',
# 'tbp_lv_Hext',
# 'tbp_lv_L',
# 'tbp_lv_Lext',
# 'tbp_lv_area_perim_ratio',
# 'tbp_lv_color_std_mean',
# 'tbp_lv_deltaA',
# 'tbp_lv_deltaB',
# 'tbp_lv_deltaL',
# 'tbp_lv_deltaLB',
# 'tbp_lv_eccentricity',
# 'tbp_lv_minorAxisMM',
# 'tbp_lv_perimeterMM',
# 'tbp_lv_radial_color_std_max',
# 'tbp_lv_stdL',
# 'tbp_lv_stdLExt',
# 'tbp_lv_symm_2axis',
# 'tbp_lv_symm_2axis_angle',
# 'tbp_lv_x',
# 'tbp_lv_y',
# 'tbp_lv_z',
# 'patient_id',
# 'image_type',
# 'tbp_tile_type',
# 'anatom_site_general',
# 'tbp_lv_location',
# 'attribution',
# 'copyright_license',
# 'iddx_full',
# 'iddx_2',
# 'iddx_3',
# 'iddx_4',
# 'iddx_5',
# 'height',
# 'width',
# 'channel',
# 'tbp_lv_nevi_confidence',
# 'tbp_lv_dnn_lesion_confidence'], axis=1, inplace=True)

# # Dropearemos las variables que en su mayoría están llenas de null
# df.drop(['mel_mitotic_index', 'lesion_id', 'mel_thick_mm'], axis=1, inplace=True)

# Quitamos ahora las columnas con datos faltantes
df.dropna(subset=['age_approx', 'sex'], inplace=True)

# Codificación binaria de sexo
df["sex"] = df["sex"].astype('category').cat.codes

# Set index a imagen id
df.set_index(df.columns[0], inplace=True)

# # Drop explicación de columna target
# df.drop(['iddx_1'], axis=1, inplace=True)

# Get dummies for categorial variable: "location simple"
df = pd.concat([df, pd.get_dummies(df['tbp_lv_location_simple'], dtype=int)], axis=1)
df.drop(['tbp_lv_location_simple'], axis=1, inplace=True)

In [ ]:
df['target'].value_counts()

target
0    9607
1     381
Name: count, dtype: int64

In [ ]:
df.head(5)

,target,age_approx,sex,clin_size_long_diam_mm,tbp_lv_areaMM2,tbp_lv_deltaLBnorm,tbp_lv_norm_border,tbp_lv_norm_color,Head & Neck,Left Arm,Left Leg,Right Arm,Right Leg,Torso Back,Torso Front
isic_id,,,,,,,,,,,,,,,
ISIC_8140686,0,70.0,0,2.85,4.240946,6.484014,3.409160,1.608265,0,0,1,0,0,0,0
ISIC_8374103,0,65.0,0,2.09,2.514543,7.706197,2.239681,2.190920,0,0,0,0,1,0,0
ISIC_7054001,0,60.0,1,2.70,3.640458,5.753112,2.897936,1.175670,0,0,0,0,0,1,0
ISIC_7511473,0,55.0,0,6.50,26.158754,8.873213,3.962258,8.242360,0,1,0,0,0,0,0
ISIC_2831112,0,50.0,0,2.68,2.852318,5.052441,3.955003,0.000000,0,0,0,0,0,0,1


#### Image data

Transform image data into a vector.

In [ ]:
# Configuration: Images Directory
image_dir = "cancer_cropped"  # folder with all the images
all_image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith('.jpg')]

# Function for loading and processing images
def load_images(paths):
    images = []
    for path in paths:
        img = io.imread(path)
        img = img_as_float(img)  # Convert to float in range [0, 1]
        f_name = os.path.splitext(os.path.basename(path))[0]
        images.append((f_name, img))
    return images

images = load_images(all_image_paths)

hog_dataset = []
# Processing and visualization example
for fname, img in images:
  fd = hog(
      orientations = 8, # EDA fueron 9
      pixels_per_cell=(8, 8),
      cells_per_block=(2, 2),
      block_norm='L2-Hys',
      visualize=False,
      channel_axis=-1,
      feature_vector=True,
      image=img
  )
  # Append a dictionary with filename and HOG features
  hog_dataset.append({"filename": fname, "hog_features": fd})

# Convert to DataFrame
hog_df = pd.DataFrame(hog_dataset)

# Explode the HOG features into individual columns
hog_features_df = pd.DataFrame(hog_df["hog_features"].tolist())
hog_df = pd.concat([hog_df[["filename"]], hog_features_df], axis=1)
hog_df.set_index('filename', inplace=True)

In [ ]:
hog_df.set_index('filename', inplace=True)
hog_df

,0,1,2,3,4,5,6,7,8,9,...,3862,3863,3864,3865,3866,3867,3868,3869,3870,3871
filename,,,,,,,,,,,,,,,,,,,,,
ISIC_0052026,0.251178,0.143841,0.086895,0.105836,0.214130,0.139083,0.243413,0.042968,0.251178,0.251178,...,0.108656,0.126701,0.261826,0.183349,0.220199,0.185911,0.137192,0.147383,0.094428,0.188792
ISIC_0073665,0.219974,0.028937,0.121711,0.051951,0.234962,0.051035,0.153254,0.065945,0.234962,0.143958,...,0.233297,0.233297,0.094139,0.149026,0.233297,0.090741,0.233297,0.139716,0.090915,0.172699
ISIC_0078294,0.247584,0.118305,0.161855,0.071849,0.247584,0.066343,0.247584,0.165856,0.231384,0.043513,...,0.271164,0.167277,0.293607,0.157918,0.234218,0.000000,0.099521,0.063748,0.214278,0.138271
ISIC_0079472,0.250449,0.036374,0.115062,0.000000,0.081786,0.039243,0.135922,0.055700,0.239770,0.045321,...,0.251837,0.126587,0.094366,0.212499,0.198773,0.230445,0.251837,0.084414,0.169828,0.075038
ISIC_0079627,0.264382,0.053172,0.129548,0.000000,0.128408,0.042538,0.052081,0.039218,0.264382,0.062196,...,0.116052,0.075388,0.245154,0.008422,0.030593,0.031116,0.094497,0.081827,0.103990,0.245154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISIC_9996602,0.063571,0.034945,0.037618,0.268682,0.268682,0.268682,0.140339,0.029559,0.248431,0.156000,...,0.234076,0.141216,0.176965,0.234076,0.074570,0.089275,0.234076,0.200754,0.234076,0.182147
ISIC_9997576,0.222041,0.047939,0.047939,0.059607,0.247695,0.247695,0.247695,0.141298,0.247695,0.063608,...,0.138715,0.216952,0.075290,0.216952,0.153607,0.114955,0.216952,0.155984,0.213536,0.102586
ISIC_9997609,0.181195,0.153180,0.230001,0.122054,0.205135,0.230001,0.182120,0.230001,0.202381,0.113955,...,0.157519,0.045235,0.222268,0.138452,0.238483,0.065121,0.238483,0.153384,0.103317,0.115311


#### Data concat

In [ ]:
full_data = df.merge(hog_df, how='inner', left_index=True, right_index=True)
full_data

,target,age_approx,sex,clin_size_long_diam_mm,tbp_lv_areaMM2,tbp_lv_deltaLBnorm,tbp_lv_norm_border,tbp_lv_norm_color,Head & Neck,Left Arm,...,3862,3863,3864,3865,3866,3867,3868,3869,3870,3871
ISIC_8140686,0,70.0,0,2.85,4.240946,6.484014,3.409160,1.608265,0,0,...,0.052720,0.126218,0.083155,0.047463,0.196677,0.222441,0.222441,0.222441,0.101143,0.180848
ISIC_8374103,0,65.0,0,2.09,2.514543,7.706197,2.239681,2.190920,0,0,...,0.089568,0.125013,0.052988,0.244796,0.244796,0.233016,0.161569,0.154181,0.169760,0.150018
ISIC_7054001,0,60.0,1,2.70,3.640458,5.753112,2.897936,1.175670,0,0,...,0.238493,0.017377,0.187781,0.225031,0.238493,0.087167,0.238493,0.238493,0.238493,0.061645
ISIC_7511473,0,55.0,0,6.50,26.158754,8.873213,3.962258,8.242360,0,1,...,0.186592,0.105298,0.240384,0.238608,0.178523,0.133874,0.231347,0.240384,0.132137,0.201423
ISIC_2831112,0,50.0,0,2.68,2.852318,5.052441,3.955003,0.000000,0,0,...,0.232511,0.244991,0.125189,0.115972,0.103017,0.086995,0.139099,0.244991,0.122447,0.029989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISIC_9877311,1,50.0,0,8.16,24.544943,8.754266,3.516098,6.918433,0,0,...,0.245939,0.206119,0.167330,0.021147,0.092517,0.050929,0.245939,0.154611,0.245939,0.163645
ISIC_9941636,1,60.0,0,5.65,15.462560,4.213853,3.493887,2.577602,0,0,...,0.018429,0.030061,0.135104,0.281469,0.281469,0.281469,0.119736,0.042604,0.057788,0.018554
ISIC_9972649,1,60.0,1,8.51,25.689620,8.661592,6.314910,10.000000,0,0,...,0.250007,0.250007,0.250007,0.093198,0.171550,0.218747,0.119567,0.177443,0.238401,0.201234
ISIC_9972877,1,65.0,0,12.08,77.857010,9.979303,2.775128,6.855612,0,0,...,0.067431,0.070010,0.042909,0.232248,0.256121,0.256121,0.256121,0.033775,0.192525,0.053402


In [ ]:
X = full_data.drop('target', axis=1)
y = full_data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Model Grid Search

In [ ]:
def save_grid_search_results(results_df, filename='grid_search_results.csv'):
    # Sort results by mean test score
    sorted_results = results_df.sort_values('mean_test_score', ascending=False)

    # Select and rename relevant columns
    columns_to_save = [
        'param_n_estimators', 'param_learning_rate', 'param_max_depth',
        'param_min_child_weight', 'param_subsample',
        'param_colsample_bytree', 'param_scale_pos_weight',
        'mean_test_score', 'std_test_score'
    ]

    # Save to CSV
    sorted_results[columns_to_save].to_csv(filename, index=False)
    print(f"Grid search results saved to {filename}")

In [ ]:
model = xgb.XGBClassifier(
    random_state=42,
    use_label_encoder=False
)

# Define parameter grid
param_grid = {
    # Number of boosting rounds
    'n_estimators': [50, 100, 200],

    # Learning rate
    'learning_rate': [0.01, 0.1, 0.3],

    # Tree depth
    'max_depth': [3, 5, 7],

    # Minimum child weight
    'min_child_weight': [1, 3, 5],

    # Subsample ratio of the training instances
    'subsample': [0.6, 0.8, 1.0],

    # Subsample ratio of columns when constructing each tree
    'colsample_bytree': [0.6, 0.8, 1.0],

    # Balance positive and negative weights
    'scale_pos_weight': [1, 2, 3]
}

# Create custom scorer focused on recall
recall_scorer = make_scorer(recall_score)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring=recall_scorer,  # Optimize for recall
    n_jobs=-1,  # Use all available cores
    verbose=2  # Print detailed information
)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(X_val)

# Performance metrics
from sklearn.metrics import classification_report, confusion_matrix

print("Best Parameters:", grid_search.best_params_)
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Create a results DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

In [ ]:
save_grid_search_results(results_df)

### Modelo con hiperparámetros

In [ ]:
# Model Training
# Define model parameters
model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.001,
        max_depth=8,
        random_state=42,
        n_jobs=-1,
        verbosity=2
)

# Fit the model
model.fit(X_train, y_train)

C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dty

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=<function recall_score at 0x000001EBFEDD6830>,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.001, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=42, ...)

### Model Evaluation

In [ ]:
# Model Evaluation
# Make predictions
y_pred = model.predict(X_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dty

In [ ]:
print(accuracy, precision, recall, f1)

0.960960960960961 0.47058823529411764 0.21052631578947367 0.2909090909090909


### Feature Importance

In [ ]:
# 4. Feature Importance
def get_feature_importance(model):
    """
    Extract and visualize feature importance

    Parameters:
    model (XGBoost model): Trained model

    Returns:
    Pandas DataFrame of feature importances
    """
    feature_importance = model.feature_importances_
    feature_names = model.feature_names_in_

    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)

    return importance_df

# Extract and visualize feature importance
importance_df = get_feature_importance(model)
importance_df

,feature,importance
3,tbp_lv_areaMM2,0.043068
7,Head & Neck,0.017720
1980,1966,0.016583
2441,2427,0.015680
2254,2240,0.015280
...,...,...
1345,1331,0.000000
1346,1332,0.000000
1347,1333,0.000000
1348,1334,0.000000


### ROC

In [ ]:
def plot_roc_curve(model, X_test, y_test):
    # Predict probabilities for the positive class
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange',
             label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')

    # Customize the plot
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")

    # Display additional metrics
    print(f"AUC Score: {roc_auc:.4f}")

    plt.show()

# Generate ROC Curve
plot_roc_curve(model, X_test, y_test)